In [96]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from pandas.tseries.offsets import DateOffset

In [97]:
df1 = pd.read_csv('idx_cleaned_data1.csv');
df2 = pd.read_csv('idx_cleaned_data2.csv');
df3 = pd.read_csv('idx_cleaned_data3.csv');
df = pd.concat([df1, df2, df3], ignore_index=True);
df.columns

/var/folders/8_/6hpk2y3x57s3wdv38bd9rprm0000gn/T/ipykernel_4345/3780464941.py:2: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('idx_cleaned_data2.csv');
/var/folders/8_/6hpk2y3x57s3wdv38bd9rprm0000gn/T/ipykernel_4345/3780464941.py:3: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  df3 = pd.read_csv('idx_cleaned_data3.csv');


Index(['BuyerAgentAOR', 'Flooring', 'ViewYN', 'PoolPrivateYN', 'CloseDate',
       'ClosePrice', 'Latitude', 'Longitude', 'UnparsedAddress',
       'PropertyType', 'LivingArea', 'DaysOnMarket', 'BuyerOfficeName',
       'BuyerAgentMlsId', 'BuyerAgentFirstName', 'BuyerAgentLastName',
       'AssociationFeeFrequency', 'MLSAreaMajor', 'CountyOrParish',
       'MlsStatus', 'ElementarySchool', 'AttachedGarageYN', 'ParkingTotal',
       'PropertySubType', 'LotSizeAcres', 'SubdivisionName', 'BuyerOfficeAOR',
       'YearBuilt', 'StreetNumberNumeric', 'BathroomsTotalInteger', 'City',
       'BuildingAreaTotal', 'BedroomsTotal', 'ContractStatusChangeDate',
       'PurchaseContractDate', 'ListingContractDate', 'StateOrProvince',
       'MiddleOrJuniorSchool', 'FireplaceYN', 'Stories', 'HighSchool',
       'Levels', 'LotSizeArea', 'MainLevelBedrooms', 'NewConstructionYN',
       'GarageSpaces', 'HighSchoolDistrict', 'PostalCode', 'AssociationFee',
       'LotSizeSquareFeet', 'Group'],
      dtype

In [98]:
df['CloseDate'] = pd.to_datetime(df['CloseDate'], yearfirst=True);

last_date = df['CloseDate'].max();
test_date = last_date-DateOffset(months=1);
train_date = test_date-DateOffset(months=6);

train_df = df[(df['CloseDate'] > train_date) & (df['CloseDate'] < test_date)];
test_df = df[(df['CloseDate'] >= test_date)];
test_df = test_df[(test_df['ClosePrice'] > test_df['ClosePrice'].quantile(0.05)) & (test_df['ClosePrice'] < test_df['ClosePrice'].quantile(0.95))]

train_y = train_df['ClosePrice'];
test_y = test_df['ClosePrice'];

In [99]:
#ran into some issues with flooring, levels - will properly onehotencode
drop_cols = ['Flooring', 'Levels', 'UnparsedAddress', 'PostalCode', 'City', 'StreetNumberNumeric', 'PropertyType', 'PropertySubType', 'ClosePrice', 'CloseDate', 'DaysOnMarket', 'BuyerOfficeName', 'BuyerAgentMlsId', 'BuyerAgentFirstName', 'BuyerAgentLastName', 'BuyerAgentAOR', 'BuyerOfficeAOR', 'ContractStatusChangeDate', 'PurchaseContractDate', 'MLSAreaMajor', 'CountyOrParish', 'ElementarySchool', 'SubdivisionName', 'ListingContractDate', 'HighSchool', 'HighSchoolDistrict', 'StateOrProvince', 'MiddleOrJuniorSchool'];
train_X = train_df.drop(columns=drop_cols);
test_X = test_df.drop(columns=drop_cols);

In [100]:
numeric_cols = list(train_X.dtypes[(train_X.dtypes == 'float64') | (train_X.dtypes == 'int')].index);
categorical_cols = list(train_X.dtypes[train_X.dtypes == 'object'].index);
ct = ColumnTransformer(
    [("StandardScaler", StandardScaler(), numeric_cols),
     ("OneHotEncode", OneHotEncoder(), categorical_cols)]);
train_X = ct.fit_transform(train_X);
test_X = ct.transform(test_X);

In [101]:
def calculate_mdape(model, test_X, test_y):
    return np.median(np.divide(np.abs(model.predict(test_X)-test_y), test_y))*100;

In [102]:
olsReg = LinearRegression().fit(train_X, train_y);
print("R^2: "+str(olsReg.score(test_X, test_y)));
print("MdAPE: "+str(calculate_mdape(olsReg, test_X, test_y)));

R^2: -0.16626391080601532
MdAPE: 60.51198443395312


In [103]:
ridgeReg = Ridge().fit(train_X, train_y);
print("R^2: "+str(ridgeReg.score(test_X, test_y)));
print("MdAPE: "+str(calculate_mdape(ridgeReg, test_X, test_y)));

R^2: -0.1661898001912272
MdAPE: 60.514789697164694


In [106]:
lassoReg = Lasso().fit(train_X, train_y);
print("R^2: "+str(lassoReg.score(test_X, test_y)));
print("MdAPE: "+str(calculate_mdape(lassoReg, test_X, test_y)));

R^2: -0.1662505724579666
MdAPE: 60.51458874411375


/Library/Frameworks/Python.framework/Versions/3.14/lib/python3.14/site-packages/sklearn/linear_model/_coordinate_descent.py:716: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.522e+17, tolerance: 3.763e+14
  model = cd_fast.enet_coordinate_descent(
